In [1]:
import tensorflow as tf
##Select GPU 0 or 1
GPU_INDEX = 1
GPU_MEM = 0.95
###GPU Configuration
gpus = tf.config.experimental.list_physical_devices('GPU')
gpu = '/gpu:'+ str(GPU_INDEX)
###Set Memory limit (DO NOT OVER 5G!!!)
tf.config.experimental.set_virtual_device_configuration(gpus[GPU_INDEX], 
                [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=GPU_MEM*10*1024)])
##WARNING!!!
'''You have to put some critical codes (e.g. Create ANN and fit)  
    inside the with statement(e.g. with tf.device(gpu):)!!!''';

In [2]:
from tensorflow.keras.datasets import cifar10
from tensorflow import keras

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train/255
x_test = x_test/255
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [3]:
from spkeras.models import cnn_to_snn
from spkeras.utils import save_pickle, load_pickle

from tensorflow.keras.models import load_model

with tf.device(gpu):
    cnn_mdl = load_model('cifar10vgg_bn.h5')

In [4]:
timesteps = [256]

thresholding = [0,0.5]
scaling_factor = [1]

with tf.device(gpu):
    cnn_result = []
    snn_result = []

    _, cnn_acc = cnn_mdl.evaluate(x_test,y_test)
    snn_mdl = cnn_to_snn(amp_factor=100000,method=0)(cnn_mdl,x_train)
    
    result = []
    for s1 in thresholding:
        _result = []
        for s2 in scaling_factor:
            __result = []
            for t in timesteps:
                _,acc = snn_mdl.evaluate(x_test,y_test,timesteps=t,
                                            thresholding=s1,                                        
                                            scaling_factor=s2)
                __result.append(acc)
            _result.append(__result)
            
        result.append(_result)
        
    cnn_result.append(cnn_acc)
    snn_result.append(result)   
                 
        
#    save_pickle(snn_result,'mdl_bn'+'_snn_result','../result/')
#    save_pickle(cnn_result,'mdl_bn'+'_cnn_result','../result/')

313/313 [==============================] - 2s 7ms/step - loss: 0.4384 - accuracy: 0.9276
{'timesteps': 256, 'thresholding': 0.5, 'amp_factor': 100000, 'signed_bit': 0, 'spike_ext': 0, 'use_bias': True, 'scaling_factor': 1, 'method': 0}
Start Converting...
Extracting Lambda...
2/60Activation
6/60Activation
7/60AveragePooling2D
10/60Activation
14/60Activation
15/60AveragePooling2D
18/60Activation
22/60Activation
26/60Activation
27/60AveragePooling2D
30/60Activation
34/60Activation
38/60Activation
39/60AveragePooling2D
42/60Activation
46/60Activation
50/60Activation
51/60AveragePooling2D
56/60Activation
59/60Activation
maximum activations: [1.0, 29.562387, 38.42166, 17.9191, 13.923333, 19.42382, 11.905245, 8.916558, 13.712846, 13.780908, 8.165588, 12.632113, 13.6503315, 13.902915, 9.4377165, 8.193663, 6.443161, 5.4482145, 4.2760696, 2.4774842, 19.505877]
Building new model...
spikeforward_0_threshold: 2956238.7466430664
spikeforward_1_threshold: 129968.05965209183
spikeforward_2_threshold

In [ ]:
with tf.device(gpu):
    snn_mdl.evaluate(x_test,y_test,timesteps=256,scaling_factor=0.98)

In [ ]:
timesteps = [16,32,48,64,80,96,112,128,144,160,176,192,208,224,240,256]

thresholding = [0,0.5]
scaling_factor = [1]

with tf.device(gpu):
    cnn_result = []
    snn_result = []

    _, cnn_acc = cnn_mdl.evaluate(x_test,y_test)
    snn_mdl = cnn_to_snn(method=0)(cnn_mdl,x_train)
    
    result = []
    for s1 in thresholding:
        _result = []
        for s2 in scaling_factor:
            __result = []
            for t in timesteps:
                _,acc = snn_mdl.evaluate(x_test,y_test,timesteps=t,
                                            thresholding=s1,                                        
                                            scaling_factor=s2)
                __result.append(acc)
            _result.append(__result)
            
        result.append(_result)
        
    cnn_result.append(cnn_acc)
    snn_result.append(result)   
                 
        
    save_pickle(snn_result,'mdl_bn'+'_snn_result','../result/')
    save_pickle(cnn_result,'mdl_bn'+'_cnn_result','../result/')

In [ ]:
timesteps = [16,32,48,64,80,96,112,128,144,160,176,192,208,224,240,256]
thresholding = [0.5]
scaling_factor = [1]
bit = [7,8,9,10]

with tf.device(gpu):
#with mirrored_strategy.scope():
    cnn_result = []
    snn_result = []

    _, cnn_acc = cnn_mdl.evaluate(x_test,y_test)
    cnn_result.append(cnn_acc)
    result_=[]
    for b in bit:
        snn_mdl = cnn_to_snn(signed_bit=b)(cnn_mdl,x_train)
        
        result = []
        for s1 in thresholding:
            _result = []
            for s2 in scaling_factor:
                __result = []
                for t in timesteps:
                    _,acc = snn_mdl.evaluate(x_test,y_test,timesteps=t,
                                                thresholding=s1,                                        
                                                scaling_factor=s2,
                                                spike_ext=0)
                    __result.append(acc)
                _result.append(__result)
                
            result.append(_result)
        result_.append(result)
            
    snn_result.append(result_)   
                 
        
    save_pickle(snn_result,'mdl_bn_bit'+'_snn_result','../result/')
    save_pickle(cnn_result,'mdl_bn_bit'+'_cnn_result','../result/')

In [ ]:
timesteps = [16,32,48,64,80,96,112,128,144,160,176,192,208,224,240,256]

thresholding = [0,0.5]
scaling_factor = [1]

import numpy as np

    
with tf.device(gpu):
    cnn_result = []
    snn_result = []
    snn_result_ = []
    
    _, cnn_acc = cnn_mdl.evaluate(x_test,y_test)
    snn_mdl = cnn_to_snn()(cnn_mdl,x_train)
    
    result = []
    result_ = []
    shape,Neuros = snn_mdl.NeuronNumbers(mode=1)

    for s1 in thresholding:
        _result = []
        _result_ = []
        for s2 in scaling_factor:
            __result = []
            __result_ = []
            for t in timesteps:
                l,N = snn_mdl.SpikeCounter(x_train,timesteps=t,
                                            thresholding=s1,                                        
                                            scaling_factor=s2)
                #Count operations 
                inp_spikes = x_train*t
                inp_spikes = np.floor(inp_spikes)
                inp_spikes = (np.sum(inp_spikes)/np.prod(x_train.shape))*np.prod(shape[0])
                inp_spikes = np.ceil(inp_spikes)
                
                a = []
                for k in range(len(Neuros)):
                    if k == len(Neuros) - 1:
                        continue
                    _a = N[k]/Neuros[k]*np.prod(shape[k+1])
                    a.append(_a)
                    
                __result_.append(np.max(l))
                __result.append(np.floor(np.sum(a+inp_spikes)))
            _result.append(__result)
            _result_.append(__result_)
        result.append(_result)
        result_.append(_result_)
        
    snn_result.append(result) 
    snn_result_.append(result)
                 
    save_pickle(snn_result,'mdl_bn_spikes'+'_snn_result','../result/')
    save_pickle(snn_result_,'mdl_bn_max_train'+'_snn_result','../result/')        